In [1]:
import pandas as pd
import os

### Lista de arquivos para leitura

Criando uma lista com os arquivos CSV baixados no DataSAS utilizando o módulo os.

**link para baixar arquivos:** https://sistemas.anac.gov.br/sas/downloads/view/frmDownload.aspx  
**Tema:** "Tarifas Transporte Aéreo Passageiros Domésticos"

Usei o os.listdir para que retorne uma lista contendo os nomes das entradas no diretório passado como caminho. O objetivo disso é possibilitar a a concatenação através da abertura por nome dos arquivos.

Mas como podem ter arquivos diferentes dos CSV que é o que interessa, utilizei um *for* com um *if* e o método *'endwith'* para buscar apenas os arquivos terminados em *'.CSV'*.

Posteriomente adicionei esses arquivos numa lista chamada *'dir_list'*.

In [2]:
dir_list = []
for l in os.listdir("C:\workspace\dash-voos-sbrf\datasets"):
    if l.endswith(".CSV"):
        dir_list.append(l)
print(dir_list)

['201801.CSV', '201802.CSV', '201803.CSV', '201804.CSV', '201805.CSV', '201806.CSV', '201807.CSV', '201808.CSV', '201809.CSV', '201810.CSV', '201811.CSV', '201812.CSV', '201901.CSV', '201902.CSV', '201903.CSV', '201904.CSV', '201905.CSV', '201906.CSV', '201907.CSV', '201908.CSV', '201909.CSV', '201910.CSV', '201911.CSV', '201912.CSV', '202001.CSV', '202002.CSV', '202003.CSV', '202004.CSV', '202005.CSV', '202006.CSV', '202007.CSV', '202008.CSV', '202009.CSV', '202010.CSV', '202011.CSV', '202012.CSV', '202101.CSV', '202102.CSV', '202103.CSV', '202104.CSV', '202105.CSV', '202106.CSV', '202107.CSV', '202108.CSV', '202109.CSV', '202110.CSV', '202111.CSV', '202112.CSV', '202201.CSV', '202202.CSV', '202203.CSV', '202204.CSV', '202205.CSV', '202206.CSV', '202207.CSV', '202208.CSV']


### Criando dum DataFrame

Agora criei um DataFrame vazio passando apenas os nomes das colunas para posteriormente recebera concatenação dos arquivos que são mensais.

In [3]:
df_sbrf = pd.DataFrame(columns=['ANO','MES','EMPRESA','ORIGEM','DESTINO','TARIFA','ASSENTOS'])

### Concatenando os arquivos

Criei um *for* para concatenar os arquivos mensais no DataFrame *'df_sbrf'* criado para receber os vôos com origem no aeroporto do Recife(SBRF).

Para isso criei:
- Um DataFrame temporario(*df_temp*);
- Usei o *'.loc'* para buscar na coluna 'ORIGEM' linhas com o conteúdo 'SBRF'; 
- Então usei o *'pd.concat'* para concatenar o conteúdo filtrado pelo *'.loc'* ao DataFrame *'df_sbrf'*.

In [ ]:
for c in range(len(dir_list)):
    df_temp = pd.read_csv(dir_list[c], sep=';')
    df_temp = df_temp.loc[(df_temp.ORIGEM == 'SBRF')]
    df_sbrf = pd.concat([df_sbrf, df_temp], ignore_index=True)

### UnicodeDecodeError

Ao rodar o código retornou o erro: **UnicodeDecodeError: 'utf-8' codec can't decode byte 0xea in position 12: invalid continuation byte**

O erro foi causado possivelmente pelo Python estar pegando o enconding local da máquina e este não ser utf-8.

A solução é declarar o enconding diretamente na codificação da seguinte forma:  
        df_temp = pd.read_csv(dir_list[c], encoding='utf-8', sep=';')

Porém a solução com o encoding *encoding='utf-8'* não resolveu o problema, então substituí pelo *encoding='ISO-8859-1'*. Daí este funcionou.

Na prática isto aconteceu porque alguns arquivos tinha cabeçalhos diferentes utilizando acentuações, detalhe que só percebi mais a frente.

In [ ]:
for c in range(len(dir_list)):
    df_temp = pd.read_csv(dir_list[c], encoding='ISO-8859-1', sep=';')
    df_temp = df_temp.loc[(df_temp.ORIGEM == 'SBRF')]
    df_sbrf = pd.concat([df_sbrf, df_temp], ignore_index=True)

### AttributeError

Ao rodar o código novamente ele aprensentou um novo erro: **AttributeError: 'DataFrame' object has no attribute 'ORIGEM'**

Então utilizei um *'try'* para testar os arquivos que estavam retornando este erro, com o *'except'* usando o *print* para mostrar estes arquivos na .

Ao verificá-los abrindo os primeiros deles com o bloco de notas percebi que eles apresentavam o cabeçalho diferente do que eu tinha verificado anteriormente, por isso não encontravam a coluna 'ORIGEM' e por causa destas colunas vinha dando o *UnicodeDecodeError*.

In [96]:
for c in range(len(dir_list)):
    try:
        df_temp = pd.read_csv(dir_list[c], encoding='ISO-8859-1', sep=';')
        df_temp = df_temp.loc[(df_temp.ORIGEM == 'SBRF')]
        df_sbrf = pd.concat([df_sbrf, df_temp], ignore_index=True)
    except:
        print(f'erro no arquivo: {dir_list[c]}')

erro no arquivo: 201802.CSV
erro no arquivo: 201803.CSV
erro no arquivo: 201804.CSV
erro no arquivo: 201805.CSV
erro no arquivo: 201806.CSV
erro no arquivo: 201807.CSV
erro no arquivo: 201808.CSV
erro no arquivo: 201810.CSV
erro no arquivo: 201811.CSV
erro no arquivo: 201812.CSV
erro no arquivo: 201903.CSV
erro no arquivo: 201912.CSV
erro no arquivo: 202001.CSV
erro no arquivo: 202002.CSV
erro no arquivo: 202003.CSV
erro no arquivo: 202005.CSV
erro no arquivo: 202006.CSV
erro no arquivo: 202007.CSV
erro no arquivo: 202008.CSV
erro no arquivo: 202009.CSV
erro no arquivo: 202010.CSV
erro no arquivo: 202011.CSV
erro no arquivo: 202012.CSV
erro no arquivo: 202102.CSV
erro no arquivo: 202103.CSV
erro no arquivo: 202104.CSV
erro no arquivo: 202106.CSV
erro no arquivo: 202107.CSV
erro no arquivo: 202108.CSV
erro no arquivo: 202112.CSV
erro no arquivo: 202201.CSV
erro no arquivo: 202202.CSV
erro no arquivo: 202203.CSV
erro no arquivo: 202204.CSV


### Solução testada

A solução que tentei foi usar *df_temp.columns = ['ANO','MES','EMPRESA','ORIGEM','DESTINO','TARIFA','ASSENTOS']* para alterar os nomes das colunas e torná-las iguais as demais.

In [ ]:
for c in range(len(dir_list)):
    df_temp = pd.read_csv(dir_list[c], encoding='ISO-8859-1', sep=';')
    df_temp.columns = ['ANO','MES','EMPRESA','ORIGEM','DESTINO','TARIFA','ASSENTOS']
    df_temp = df_temp.loc[(df_temp.ORIGEM == 'SBRF')]
    df_sbrf = pd.concat([df_sbrf, df_temp], ignore_index=True)

### ValueError

Ao rodar o código ele aprensentou um novo erro: **ValueError: Length mismatch: Expected axis has 8 elements, new values have 7 elements**

Utilizei novamento o *'try'* e o *'except'* para mostrar os arquivos com problemas.

In [98]:
for c in range(len(dir_list)):
    try:
        df_temp = pd.read_csv(dir_list[c], encoding='ISO-8859-1', sep=';')
        df_temp.columns = ['ANO','MES','EMPRESA','ORIGEM','DESTINO','TARIFA','ASSENTOS']
        df_temp = df_temp.loc[(df_temp.ORIGEM == 'SBRF')]
        df_sbrf = pd.concat([df_sbrf, df_temp], ignore_index=True)
    except:
        print(f'erro no arquivo: {dir_list[c]}')

erro no arquivo: 202112.CSV
erro no arquivo: 202204.CSV


### Solução testada:

Ao abrir os dois arquivos com problemas notei que eles apresentavam uma primeira coluna com índice, diferente dos demais que não vinham com esta coluna.

Como a coluna *'índice'* não era nomeada testei utilizar uma linha de código *df_temp.drop("", axis=1, inplace=True)* para remover a primeira coluna.

In [ ]:
for c in range(len(dir_list)):
    try:
        df_temp = pd.read_csv(dir_list[c], encoding='ISO-8859-1', sep=';')
        df_temp.columns = ['ANO','MES','EMPRESA','ORIGEM','DESTINO','TARIFA','ASSENTOS']
        df_temp = df_temp.loc[(df_temp.ORIGEM == 'SBRF')]
        df_sbrf = pd.concat([df_sbrf, df_temp], ignore_index=True)
    except ValueError:
        df_temp = pd.read_csv(dir_list[c], encoding='ISO-8859-1', sep=';')
        df_temp.drop("", axis=1, inplace=True)
        df_temp.columns = ['ANO','MES','EMPRESA','ORIGEM','DESTINO','TARIFA','ASSENTOS']
        df_temp = df_temp.loc[(df_temp.ORIGEM == 'SBRF')]
        df_sbrf = pd.concat([df_sbrf, df_temp], ignore_index=True)

### KeyError

Ao rodar o código ele aprensentou mais um novo erro: **KeyError: "[''] not found in axis"**

Ele apresentou este erro pois não consegui encontrar uma coluna com o nome ''. A solução que criei foi renomear a colunar para posteriormente excluí-la.

Então utilizei o método *df_temp.columns.values[0] = 'excluir'* para renomear a coluna de nome vazio para *excluir* e depois utilizei a *df_temp.drop('excluir', axis=1, inplace=True)* novamente.

In [111]:
for c in range(len(dir_list)):
    try:
        df_temp = pd.read_csv(dir_list[c], encoding='ISO-8859-1', sep=';')
        df_temp.columns = ['ANO','MES','EMPRESA','ORIGEM','DESTINO','TARIFA','ASSENTOS']
        df_temp = df_temp.loc[(df_temp.ORIGEM == 'SBRF')]
        df_sbrf = pd.concat([df_sbrf, df_temp], ignore_index=True)
    except ValueError:
        df_temp = pd.read_csv(dir_list[c], encoding='ISO-8859-1', sep=';')
        df_temp.columns.values[0] = 'excluir'
        df_temp.drop('excluir', axis=1, inplace=True)
        df_temp.columns = ['ANO','MES','EMPRESA','ORIGEM','DESTINO','TARIFA','ASSENTOS']
        df_temp = df_temp.loc[(df_temp.ORIGEM == 'SBRF')]
        df_sbrf = pd.concat([df_sbrf, df_temp], ignore_index=True)

### Concluída a etapa

Este então foi o código final para concatenar os arquivos de vendas de passagens mensais baixados no DataSAS da ANAC.

In [112]:
df_sbrf

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS
0,2018,1,AZU,SBRF,SBAE,"1007,9",3
1,2018,1,AZU,SBRF,SBAE,"1030,53",1
2,2018,1,AZU,SBRF,SBAE,"1117,9",1
3,2018,1,AZU,SBRF,SBAE,"1157,9",1
4,2018,1,AZU,SBRF,SBAE,"1350,93",1
...,...,...,...,...,...,...,...
704222,2022,8,TAM,SBRF,SWSI,"1547,01",1
704223,2022,8,TAM,SBRF,SWSI,"1597,00",6
704224,2022,8,TAM,SBRF,SWSI,"1767,01",1
704225,2022,8,TAM,SBRF,SWSI,"1831,99",1
